# Building Ships

Ships are large structures that can be built in the `industrial_complex`. They consume resources to make and must be stored in a `shipyard`. 

In [1]:
import sys, os
import pandas as pd
import altair as alt
import yaml
sys.path.append('../..')

import  ssl, asyncio

ssl._create_default_https_context = ssl._create_unverified_context
asyncio.set_event_loop_policy(asyncio.WindowsSelectorEventLoopPolicy())
import nest_asyncio
# this is required for running in a Jupyter Notebook. 

nest_asyncio.apply()

PII_cols = ['username','userguid','owner']

In [2]:
from app.objects import baseobjects
from app.objects import ships
from app.objects import population

In [3]:
from app.connectors.cmdb_graph import CosmosdbClient
c = CosmosdbClient()

executing local windows deployment
something wrong with your query: <class 'Exception'>


industrial_complex `has_buttons`, which can `build_ship`. Ships are built from `designs`, which are also in the config. 

In [18]:
ship_configurations = ships.ship_configurations
ship_configurations.keys()

dict_keys(['explanation', 'components', 'designs'])

A ships effort is the sum of it's components

In [5]:
probe = ship_configurations['designs']['probe']
probe

{'type': 'probe',
 'label': 'ship',
 'name': 'Probe',
 'description': 'One way scanning machine to observe a location. Requires ',
 'components': ['engine', 'scanner']}

# The Ship object

In [6]:
class Component(baseobjects.Baseobject):
    def __init__(self, config):
        super().__init__()
        self.label = "component"
        self.config = config
        self.type = config['type']
        self.name = config['name']
    

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.config.items():
            fund[k] = v
        fund['type'] = self.config['type']
        return fund


In [7]:
class Design(baseobjects.Baseobject):
    def __init__(self, config):
        super().__init__()
        self.label = "design"
        self.config = config
        self.type = config['type']
        self.name = config['name']
        self.design_type = config['label']
    

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.config.items():
            fund[k] = v
        fund['type'] = self.config['type']
        fund['label'] = self.label
        return fund


In [8]:
class Ship(baseobjects.Baseobject):
    def __init__(self, design, component_configurations):
        super().__init__()
        self.label = "ship"
        # TODO: Naming for ships
        self.name = design['name']
        self.type = design['type']
        self.design = Design(design)
        self.components = [Component(component_configurations[c]) for c in design['components']]
        self.stats = {}
        self.build_stats()
    
    def build_stats(self):
        for i in self.components:
            c = i.get_data()
            if 'augments_ship_stats' in c.keys():
                for k,v in c['augments_ship_stats'].items():
                    if k in self.stats.keys():
                        self.stats[k] += v
                    else:
                        self.stats[k] = v

    
    def get_upload_data(self):
        data = {'nodes':[], 'edges':[]}
        data['nodes'].append(self.design.get_data())
        for i in self.components:
            data['nodes'].append(i.get_data())
            data['edges'].append({'source':self.objid, 'target':i.objid, 'label':'has'})
        data['nodes'].append(self.get_data())
        return data

    def get_data(self):
        fund = self.get_fundimentals()
        for k,v in self.stats.items():
            fund[k] = v
        fund['type'] = self.type
        return fund



In [9]:
ship = Ship(probe, ship_configurations['components'])
ship

<ship: probe; 7176219766700; Probe>

In [10]:
ship.components

[<component: engine; 0938142536133; Engine>,
 <component: scanner; 8174853871945; Scanner>]

In [11]:
ship.stats

{'speed': 1, 'build_effort': 2, 'cost': 2}

In [12]:
ship.get_data()

{'name': 'Probe',
 'objid': '7176219766700',
 'label': 'ship',
 'speed': 1,
 'build_effort': 2,
 'cost': 2,
 'type': 'probe'}

In [13]:
pd.DataFrame(ship.get_upload_data()['nodes'])

,name,objid,label,type,description,components,augments_ship_stats,fuel,render,grants_capability,speed,build_effort,cost
0,Probe,0490982462913,design,probe,One way scanning machine to observe a location...,"[engine, scanner]",NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engine,0938142536133,component,engine,Engines for the ship. Increases the speed and ...,NaN,"{'speed': 1, 'build_effort': 1, 'cost': 1}",0.0,cone,NaN,NaN,NaN,NaN
2,Scanner,8174853871945,component,scanner,Scans the environment for resources.,NaN,"{'cost': 1, 'build_effort': 1}",NaN,ball,[scan],NaN,NaN,NaN
3,Probe,7176219766700,ship,probe,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,2.0


In [14]:
pd.DataFrame(ship.get_upload_data()['edges'])

,source,target,label
0,7176219766700,0938142536133,has
1,7176219766700,8174853871945,has


## The Fabrication process:
| Step | Description                                                                 |
|------|-----------------------------------------------------------------------------|
| 1    | User will click on the `build_ship` button in the UX.                       |
| 2    | The server will validate that the faction can build a ship.                 |
| 3    | Ajax will take the building info and create a task.                         |
| 4    | A job is created for the build.                                             |
| `app.objects.structures.build_ship`   | Upon completion, the ship is added to whichever `shipyard` exists in that faction. |

* The ajax button is here: app\templates\app\ajax\building_action.js
* The Backend Function is here: app\models\buildings.py

In [15]:
message = {'agent': {'isIdle': 'false', 'name': 'Poldiasenah Ferfenmu', 'objid': '6668897649955', 'conformity': 0.477, 'literacy': 0.456, 'aggression': 0.5, 'constitution': 0.404, 'health': 0.7, 'isIn': '4060880433595', 'industry': 0.452, 'wealth': 0.454, 'factionLoyalty': 0.525, 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'pop'}, 'action': {'type': 'fabricating', 'comment': 'Poldiasenah Ferfenmu:6668897649955 building a Probe', 'effort': 2, 'building': 'probe', 'faction_costs': 2, 'created_at': 1014, 'to_build': '{type: probe, label: ship, name: Probe, description: One way scanning machine to observe a location. Requires , components: [engine, scanner]}', 'objid': '2604129141389', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'}, 'job': {'status': 'pending', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'name': 'takingAction', 'weight': '1016', 'actionType': 'fabricating', 'created_at': '1014'}}
message

{'agent': {'isIdle': 'false',
  'name': 'Poldiasenah Ferfenmu',
  'objid': '6668897649955',
  'conformity': 0.477,
  'literacy': 0.456,
  'aggression': 0.5,
  'constitution': 0.404,
  'health': 0.7,
  'isIn': '4060880433595',
  'industry': 0.452,
  'wealth': 0.454,
  'factionLoyalty': 0.525,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'pop'},
 'action': {'type': 'fabricating',
  'comment': 'Poldiasenah Ferfenmu:6668897649955 building a Probe',
  'effort': 2,
  'building': 'probe',
  'faction_costs': 2,
  'created_at': 1014,
  'to_build': '{type: probe, label: ship, name: Probe, description: One way scanning machine to observe a location. Requires , components: [engine, scanner]}',
  'objid': '2604129141389',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'action'},
 'job': {'status': 'pending',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'name': 'takingAction',
  'weight': '1016',
  'actionType': 'fabricating',
  'created_at': '101

In [16]:
agent = message['agent']
if agent['current_design'] == 'probe':
    design_config = ship_configurations['designs']['probe']

In [17]:
design_config

{'type': 'probe',
 'label': 'ship',
 'name': 'Probe',
 'description': 'One way scanning machine to observe a location. Requires ',
 'components': ['engine', 'scanner']}

In [18]:
design_config['components']

['engine', 'scanner']

In [19]:
ship = Ship(design_config, ship_configurations['components'])
ship

<ship: probe; 0479737364650; Probe>

In [20]:
ship.stats

{'speed': 1, 'build_effort': 2, 'cost': 2}

In [21]:
agent

{'objid': '9662163071808',
 'type': 'manufacturing',
 'name': 'Industrial manufacturing complex',
 'faction_augments': '{wealth: -1}',
 'description': 'Builds ships and other large objects for the faction.',
 'has_buttons': '[build_ship]',
 'current_design': 'probe',
 'id': '9662163071808',
 'owner': '5657120867640',
 'objtype': '<->'}

## Seeing it in the function

* go into the dev environment, build a shipyard and a manufacturing plant. 
* below, increment the timer until the job is ready
* run the process action message to generate the message

In [4]:
import function_app as f

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.info("logs showing as print")

INFO:root:logs showing as print


In [5]:
action_messages = f.process_action_messages()

DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: health requirement 0.7
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: healthy_pops_query 0
INFO:root:EXOADMIN: No pops that meet the pop_health_requirement
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: Total jobs: 1
INFO:root:EXOADMIN: job instance of ACTION created: {'action': {'type': 'fabricating', 'comment': 'Poldiasenah Ferfenmu:6668897649955 building a Probe', 'effort': 2, 'building': 'probe', 'faction_costs': 2, 'created_at': 1021, 'to_build': '{type: probe, label: ship, name: Probe, description: One way scanning machine to observe a location. Requires , components: [engine, scanner]}', 'objid': '4865001438778', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'objtype': 'action'}, 'job': {'status': 'pending', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86', 'name': 'takingAction', 'weight': '1023', 'actionType': 'fabricating', 'created_at': '1021'}, 'agent': {'isIdle': 'f

In [6]:
# f.increment_timer()

In [7]:
pd.DataFrame(action_messages)

,agent,action,job
0,"{'isIdle': 'false', 'name': 'Poldiasenah Ferfe...","{'type': 'fabricating', 'comment': 'Poldiasena...","{'status': 'pending', 'userguid': '8d5b667f-b2..."
1,"{'objid': '9888651460599', 'consumes': ['organ...",consume,NaN
2,"{'objid': '9629113611489', 'consumes': ['organ...",consume,NaN
3,"{'objid': '6668897649955', 'consumes': ['organ...",consume,NaN
4,"{'objid': '6880883763526', 'consumes': ['organ...",consume,NaN
5,"{'objid': '7372021564759', 'consumes': ['organ...",consume,NaN
6,"{'objid': '2416478665710', 'consumes': ['organ...",consume,NaN
7,"{'objid': '0255814826981', 'consumes': ['organ...",consume,NaN
8,"{'replenish_rate': 10, 'volume': 1044.0, 'obji...",renew,NaN


In [8]:
message = action_messages[0]
message

{'agent': {'isIdle': 'false',
  'name': 'Poldiasenah Ferfenmu',
  'objid': '6668897649955',
  'conformity': 0.477,
  'literacy': 0.456,
  'aggression': 0.5,
  'constitution': 0.404,
  'health': 0.7,
  'isIn': '4060880433595',
  'industry': 0.452,
  'wealth': 0.454,
  'factionLoyalty': 0.525,
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'pop'},
 'action': {'type': 'fabricating',
  'comment': 'Poldiasenah Ferfenmu:6668897649955 building a Probe',
  'effort': 2,
  'building': 'probe',
  'faction_costs': 2,
  'created_at': 1021,
  'to_build': '{type: probe, label: ship, name: Probe, description: One way scanning machine to observe a location. Requires , components: [engine, scanner]}',
  'objid': '4865001438778',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'objtype': 'action'},
 'job': {'status': 'pending',
  'userguid': '8d5b667f-b225-4641-b499-73b77558ff86',
  'name': 'takingAction',
  'weight': '1023',
  'actionType': 'fabricating',
  'created_at': '102

In [9]:
data = ships.fabricate(c, message, commit=False)

INFO:root:EXOADMIN: fabricating a ship
INFO:root:EXOADMIN: possible ship designs: ['probe']
INFO:root:EXOADMIN: ship design to build: probe
INFO:root:EXOADMIN: created a ship: <ship: probe; 8197414110507; Probe>
DEBUG:asyncio:Using selector: SelectSelector
INFO:root:EXOADMIN: placing ship: 8197414110507 in shipyard:4756190697931
INFO:root:EXOADMIN: expensing a ship: not implemented
INFO:root:EXOADMIN: data created: 4 nodes and 3 edges
INFO:root:EXOADMIN: fabrication complete for ship: {'name': 'Probe', 'objid': '8197414110507', 'label': 'ship', 'speed': 1, 'build_effort': 2, 'cost': 2, 'type': 'probe', 'userguid': '8d5b667f-b225-4641-b499-73b77558ff86'}


In [10]:
pd.DataFrame(data['nodes'])

,name,objid,label,type,description,components,userguid,augments_ship_stats,fuel,render,grants_capability,speed,build_effort,cost
0,Probe,8778861527915,design,probe,One way scanning machine to observe a location...,"[engine, scanner]",8d5b667f-b225-4641-b499-73b77558ff86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Engine,8179375896921,component,engine,Engines for the ship. Increases the speed and ...,NaN,8d5b667f-b225-4641-b499-73b77558ff86,"{'speed': 1, 'build_effort': 1, 'cost': 1}",0.0,cone,NaN,NaN,NaN,NaN
2,Scanner,4943781384881,component,scanner,Scans the environment for resources.,NaN,8d5b667f-b225-4641-b499-73b77558ff86,"{'cost': 1, 'build_effort': 1}",NaN,ball,[scan],NaN,NaN,NaN
3,Probe,8197414110507,ship,probe,NaN,NaN,8d5b667f-b225-4641-b499-73b77558ff86,NaN,NaN,NaN,NaN,1.0,2.0,2.0


In [11]:
pd.DataFrame(data['edges'])

,node1,node2,label
0,8197414110507,8179375896921,has
1,8197414110507,4943781384881,has
2,8197414110507,4756190697931,isIn
